In [5]:
import pandas as pd
Dim_customer=pd.read_csv('/content/Dim_customer.csv')
Fact_invoiceline=pd.read_csv('/content/Fact_invoiceline.csv')
Dim_track=pd.read_csv('/content/Dim_track.csv')
Fact_invoice=pd.read_csv('/content/Fact_invoice.csv')

In [6]:
#Get the tables together and only the columns that we will need
invoiceline=pd.merge(Fact_invoiceline,Fact_invoice,on='invoiceid')
invoiceline=invoiceline[['invoicelineid','invoiceid','trackid','customerid']]
invoice_track=pd.merge(invoiceline,Dim_track,on='trackid')
customer_track=invoice_track[['trackid','track_name','customerid','genreid','genre_name']]

In [7]:
def top_genre(customerid): #function to get the top genre of the client
  top_2_genre=customer_track[customer_track["customerid"] == customerid]["genre_name"].value_counts().head(2)
  top_2_genre= top_2_genre.index.tolist()
  return top_2_genre

def songs_bought(customerid,top_genre): #function to get the songs that he bought from those genre
  songs_bought=customer_track[customer_track["customerid"] == customerid]
  track_list=songs_bought[songs_bought["genre_name"].isin(top_genre)]["track_name"].tolist()
  return track_list

def recomend_songs(top_genre, songs_bought):
    # Filter tracks based on the genres and exclude the songs already bought
    filtered_tracks = customer_track[customer_track["genre_name"].isin(top_genre)]
    new_tracks = filtered_tracks[~filtered_tracks["track_name"].isin(songs_bought)]

    # Group by genre and track name, then count occurrences of each track
    recomend_songs = (new_tracks.groupby("genre_name")["track_name"].value_counts().groupby(level=0).head(3))
    print("Here are some songs you might like: ")
    return recomend_songs

In [8]:
def get_customer_id(): #get the customer ID and check if the entry is a number and also if it is a valid customer
    while True:
        try:
            customer_id = input("Please, insert your Customer ID: ")
            customer_id = int(customer_id)
            if customer_id in Dim_customer["customerid"].unique():
                return customer_id
            else:
                print(f"Customer ID {customer_id} is invalid. Please try again.")
        except ValueError:
            print("Please, enter a valid number.")

In [9]:
customer_id=get_customer_id()
top_2_genre=top_genre(customer_id)
songs_bought=songs_bought(customer_id,top_2_genre)
recomend_songs(top_2_genre, songs_bought)

Please, insert your Customer ID: 19
Here are some songs you might like: 


genre_name  track_name                                      
Latin       Banditismo Por Uma Questa                           13
            Brasil                                              12
            Dezesseis                                           12
Rock        Dazed and Confused                                  17
            Polly                                               17
            Elderly Woman Behind The Counter In A Small Town    15
Name: count, dtype: int64